# Subir Videos desde Carpeta Temporal a S3

Este notebook muestra cómo utilizar la librería `video_s3_uploader.py` para subir videos desde una carpeta temporal a un bucket de S3.

In [ ]:
# Importar la librería
import video_s3_uploader as uploader
import os

## Configuración

Definir la carpeta temporal y el bucket de S3

In [ ]:
# Configurar parámetros
temp_folder = "./tmp"  # Carpeta temporal donde están los videos
bucket_name = "mi-bucket-videos"  # Nombre del bucket de S3
prefix = "videos"  # Prefijo opcional para organizar los archivos en S3

## Verificar que la carpeta temporal existe

In [ ]:
# Verificar si la carpeta temporal existe
if not os.path.exists(temp_folder):
    print(f"La carpeta {temp_folder} no existe. Creándola...")
    os.makedirs(temp_folder)
    print(f"Carpeta {temp_folder} creada. Por favor, coloca tus videos en esta carpeta.")
else:
    print(f"La carpeta {temp_folder} existe.")
    # Listar archivos en la carpeta
    files = os.listdir(temp_folder)
    video_files = [f for f in files if f.endswith(('.mp4', '.avi', '.mov', '.mkv', '.webm'))]
    print(f"Videos encontrados: {len(video_files)}")
    for video in video_files:
        print(f"  - {video}")

## Verificar y crear el bucket de S3 si es necesario

In [ ]:
# Verificar si el bucket existe
if uploader.check_bucket_exists(bucket_name):
    print(f"El bucket {bucket_name} existe.")
else:
    print(f"El bucket {bucket_name} no existe. Creándolo...")
    # Puedes especificar una región si lo deseas
    region = "us-east-1"  # Cambia esto según tu región preferida
    if uploader.create_bucket(bucket_name, region):
        print(f"Bucket {bucket_name} creado exitosamente en la región {region}.")
    else:
        print(f"Error al crear el bucket {bucket_name}.")

## Subir videos a S3

In [ ]:
# Subir videos a S3
results = uploader.upload_videos_to_s3(temp_folder, bucket_name, prefix)

# Mostrar resultados
print(f"\nResultados de la subida:")
for result in results:
    if result["status"] == "success":
        print(f"✅ {result['filename']} -> {result['s3_url']}")
    else:
        print(f"❌ {result['filename']} -> Error: {result['error']}")

# Resumen
successful = sum(1 for r in results if r["status"] == "success")
failed = sum(1 for r in results if r["status"] == "failed")
print(f"\nResumen: {successful} videos subidos exitosamente, {failed} fallidos.")

## Ejemplo de uso de los videos subidos

Una vez que los videos están en S3, puedes utilizarlos para diferentes propósitos, como procesamiento de video, análisis o integración con otros servicios de AWS.

In [ ]:
# Ejemplo: Obtener las URLs de los videos subidos
successful_uploads = [result for result in results if result["status"] == "success"]
video_urls = [result["s3_url"] for result in successful_uploads]

print("URLs de los videos subidos:")
for url in video_urls:
    print(url)